In [1]:
import numpy as np

**Constructing Kernels**

Let $K_1$, $K_2$ be (Mercer) kernels over $\mathbb{R}^n\times \mathbb{R}^n$, let $a \in \mathbb{R}^+$ be a positive real number, let $f : \mathbb{R}^n \rightarrow \mathbb{R}$ be a real-valued function, let $\phi : \mathbb{R}^n \rightarrow \mathbb{R}^d$ be a function mapping from $\mathbb{R}^n$ to $\mathbb{R}^d$, let $K_3$ be a kernel over $\mathbb{R}^d \times \mathbb{R}^d$, and let $p(x)$ be a polynomial over $x$ with *positive* coefficients. We know want to know which of the following functions are also kernels. That is, which are symmetric, positive semi-definite matrices for some finite set $\{x^{(1)},...,x^{(m)}\}$.

(a) $K(x,z) = K_1(x,z) + K_2(x,z)$

This is clearly a kernel. Using the fact that $K_1$ and $K_2$ are kernels, and therefore symmetric, we find $K_{ji} = K_{1,ji} + K_{2,ji} = K_{1,ij} + K_{2,ij} = K_{ij}$, so that $K$ is symmetric. Similarly, the fact that $z^T Kz \geq 0$, $\forall z \in \mathbb{R}^n$ follows from linearity and the positive semi-definite nature of both $K_1$ and $K_2$

(b) $K(x,z) = K_1(x,z) - K_2(x,z)$

This is not a kernel. Let $K_2 = b K_1$, $b >1$ and $b \in \mathbb{R}^+$. Then, notice that $z^T Kz = (1-b) (z^T K_1z) \leq 0$ since $K_1$ is positive semi-definite. So, in this case $K$ is negative semi-definite

(c) $K(x,z) = aK_1(x,z)$

This is a kernel. Transposition and matrix multiplication commute with scalar multiplication, and $K$ is therefore symmetric following directly from the symmetric nature of $K_1$. Since $a$ is a *positive* real, and $K_1$ is positive semi-definite, the positive semi-definite nature of $K$ follows in a similar fashion.

(d) $K(x,z) = -aK_1(x,z)$

This is *not* a kernel. In particular, notice that $z^T Kz = -az^T K_1 z \leq 0$ since $K_1$ is positive semi-definite and $a$ is a positive real. This means that $K$ is a negative semi-definite matrix and cannot be a kernel.

(e) $K(x,z) = K_1(x,z)K_2(x,z)$

This is a kernel. Recall that, as kernels, we can write $K_1(x,z) = \phi_1^T(x) \phi_1(z)$ and $K_2 = \phi_2^T(x) \phi_2(z)$. Thus, $K(x,z) = \phi_1^T(x) \phi_1(z) \phi_2^T(x) \phi_2(z) = \sum_{i,j} \phi_{1,i}(x) \phi_{1,i}(z) \phi_{2,j}(x) \phi_{2,j}(z)$. If we define $\Phi = \phi_1 \otimes \phi_2 \rightarrow \Phi_{i,j} = \phi_{1,i} \phi_{2,j}$, we then have $K(x,z) = \Phi(x)^T \Phi$, so $K$ has the appropriate form to be a kernel as well.

(f) $K(x,z) = f(x)f(z)$.

This is a kernel. Since $f$ is a scalar, we can just define $\phi = f$ and $K$ trivially has the form $K(x,z) = \phi^T(x) \phi(z)$

**Kernelizing the Perceptron**

The original update rule we want to modify is $\theta := \theta + \alpha y^{(i)} x^{(i)}$ if $y^{(i)}h\left(x^{(i)}\right) < 0$, and $\theta := \theta$ otherwise. Working in the high-dimensional space, we now use the modified update rule $\theta := \theta + 
\alpha \left(y^{(i)}-h\left(\theta^T \phi\left(x^{(i)}\right)\right)\right)\phi\left(x^{(i)}\right)$. Note that this follows the same spirit as the original update rule: if $h(\phi)$ makes a correct predicition (same sign as the target), no update is made, whereas making an incorrect prediction causes $\theta$ to be moved in the right direction to push the margin $y^{(i)}\theta^T x^{(i)}$ towards a positive number.

Now, if we initialize $\theta = 0$, we will always have $\theta = \sum_{i=0}^n \beta_i \phi\left(x^{(i)}\right)$ after the first $n$ coefficients, so that predictions can be made just using the kernel: $h\left(\theta^T \phi(x)\right) = h\left(\sum_{i=0}^n \beta_i K\left(x^{(i)},x\right)\right)$.

To train the perceptron, we initialize the coefficients ${\beta_0,...\beta_m} = 0$. Then, the $(n+1)$-th term is computed using the update rule and the kernel as $\beta_{n+1} = y^{(n+1)}-h\left(\sum_{i=0}^{n} \beta_i K\left(x^{(i)},x\right)\right)$.

**Spam classification**

We'll tackle the problem of spam classification, first using Naive Bayes. First, let's make a function to read in the traiing and test data in the form of a document-word matrix, where the $(i,j)$-th element of the matrix indicates how many instances of word $j$ from our list of token occurs within document $i$.

In [2]:
def readDocWordMatrix(fileName):
    file = open(fileName,'r')
    file.readline() # meaningless header
    [nDocs,nToks] = [int(s) for s in file.readline().split()]
    docWordMat = np.zeros((nDocs,nToks))
    tokens = file.readline().split()
    categories = np.zeros(nDocs)
    for idx,line in enumerate(file):
        nums = [int(s) for s in line.split()]
        categories[idx] = nums[0]
        lastIdx = len(nums)-1
        docWordMat[idx,np.cumsum(nums[1:lastIdx:2])] = nums[2:lastIdx:2]
    file.close()
    return docWordMat,tokens,categories

Now, we compute the Naive Bayes parameters from the traiing set, using Laplace smoothing. Using the multinomial event model, the relevant parameters are then given by 

$\phi_{k|y=1} = p(x_j=k|y=1) = \frac{1+\sum 1\left\{x_j^{(i)}=k \wedge y^{(i)}=1\right\}}{|V|+\sum 1\left\{y^{(i)}=1\right\}n_i}$,

$\phi_{k|y=0} = p(x_j=k|y=0) = \frac{1+\sum 1\left\{x_j^{(i)}=k \wedge y^{(i)}=0\right\}}{|V|+\sum 1\left\{y^{(i)}=0\right\}n_i}$,

$\phi_{y=1} = p(y=1) = \frac{\sum 1\left\{y^{(i)}=1\right\}}{m}$,

where $|V|$ is the size of the vocabulary, $n_i$ is the total number of words in each training example.

In [3]:
def trainNaiveBayes(file):
    docWordMat,tokens,categories = readDocWordMatrix(file)
    numWords = np.sum(docWordMat,axis=1)
    numTokens = len(tokens)
    likelihoodSpam = np.zeros(numTokens)
    likelihoodNoSpam = np.zeros(numTokens)
    probSpam= np.sum(categories)/len(categories)
    for idx in range(numTokens):
        likelihoodSpam[idx] = (np.sum(docWordMat[categories==1,idx])+1)/(np.sum(numWords[categories==1])+numTokens)
        likelihoodNoSpam[idx] = (np.sum(docWordMat[categories==0,idx])+1)/(np.sum(numWords[categories==0])+numTokens)
    return likelihoodSpam,likelihoodNoSpam,probSpam

In [4]:
likelihoodSpam,likelihoodNoSpam,probSpam = trainNaiveBayes('MATRIX.TRAIN')

Now, we evaluate on the test set. For each document (row in the document-word matrix), we'll compute the conditional probability of seeing that word vector $x$ if the document is spam ($y = 1$) using the following formula:

$p(x|y=1) = \prod_{x_i} \left(\phi_{i|y=1}\right)^{x_i}$

and similarly for $y=0$. Then the probability of the document being spam is simply computed as 

$p(y=1|x) = \frac{p(x|y=1)\phi_{y=1}}{p(x|y=0)(1-\phi_{y=1})+p(x|y=1)\phi_{y=1}}$,

With the e-mail being classified as spam if $p(y=1|x) \geq 0.5$. 

The issue here is that most of the $\phi_{i|y=1}$ and $\phi_{i|y=0}$ are quite small, so we worry about underflow due to the product of so many small numbers. Thus, we look at the logarithm of the probability, converting the products to sums, and ignore the denominator (since it is only there for normalization). Thus, we compute

$l_1 = ln\left(p(x|y=1)\phi_{y=1}\right) = ln\left(\phi_{y=1}\right) + \sum_{x_i} x_i ln\left(\phi_{i|y=1}\right)$,

$l_0 = ln\left(p(x|y=0)\phi_{y=0}\right) = ln\left(1-\phi_{y=1}\right) + \sum_{x_i} x_i ln\left(\phi_{i|y=0}\right)$,

and classify as spam if $e^{l_1} > e^{l_0}$, or equivalently $l_1 > l_0$.

In [5]:
def testNaiveBayes(likelihoodSpam,likelihoodNoSpam,probSpam):
    docWordMatTest,tokensTest,categoriesTest = readDocWordMatrix('MATRIX.TEST')
    numCorrect = 0
    for testIdx,testLabel in enumerate(categoriesTest):
        testVec = docWordMatTest[testIdx,:]
        l1 = np.log(probSpam)+np.dot(testVec,np.log(likelihoodSpam))
        l0 = np.log(1-probSpam)+np.dot(testVec,np.log(likelihoodNoSpam))
        predictLabel = int(l1 >= l0)
        numCorrect += int(predictLabel==int(testLabel))
    print("Training error is "+str(100*(1-numCorrect/len(categoriesTest)))+"%")

In [6]:
testNaiveBayes(likelihoodSpam,likelihoodNoSpam,probSpam)

Training error is 1.6249999999999987%


We can also estimate the most likely spam words by looking at which words maximize the quantity $ln\left(\frac{\phi_{k|y=1}}{\phi_{k|y=0}}\right)$

In [7]:
docWordMat,tokens,categories = readDocWordMatrix('MATRIX.TRAIN')
wordLikelihood = np.log(likelihoodSpam/likelihoodNoSpam)
np.argmax(wordLikelihood)
sortedInds = np.argsort(wordLikelihood)[::-1]
spamTokens = [tokens[s] for s in sortedInds[0:5]]
spamTokens

['httpaddr', 'spam', 'unsubscrib', 'ebai', 'valet']

We can also see how the accuracy of the classifier converges with the amount of training data. In this case, we can see that even 50-100 documents is still quite good (~2x error compared to the full training set of over 2000 doucments), and the error has converged to that of a full training set already by ~1400 documents.

In [8]:
trainingSize = ['50','100','200','400','800','1400']
for size in trainingSize:
    likelihoodSpam,likelihoodNoSpam,probSpam = trainNaiveBayes('MATRIX.TRAIN.'+size)
    print('Training size = '+size+' documents')
    testNaiveBayes(likelihoodSpam,likelihoodNoSpam,probSpam)

Training size = 50 documents
Training error is 3.874999999999995%
Training size = 100 documents
Training error is 2.6249999999999996%
Training size = 200 documents
Training error is 2.6249999999999996%
Training size = 400 documents
Training error is 1.8750000000000044%
Training size = 800 documents
Training error is 1.749999999999996%
Training size = 1400 documents
Training error is 1.6249999999999987%
